$x1$ : número de productos fabricados de tipo 1   
$x2$ : número de productos fabricados de tipo 2   
$x3$ : número de productos fabricados de tipo 3   

Maximizar $z = 4.5x1 + 5x2 + 4x3$   
sujeto a   
$10x1 + 5x2 + 6x3 <= 600$  
$6x1 + 8x2 + 9x3 <= 600$  
$8x1 + 10x2 + 12x3 <=600$  

$x1$ : número de productos fabricados de tipo 1   
$x2$ : número de productos fabricados de tipo 2   
$x3$ : número de productos fabricados de tipo 3   
$s1$ : variable de holgura 1   
$s2$ : variable de holgura 2   
$s3$ : variable de holgura 3  

Maximizar $z - 4.5x1 - 5x2 - 4x3 = 0$  
sujeto a   
$10x1 + 5x2 + 6x3 + s1 = 600$   
$6x1 + 8x2 + 9x3 + s2 = 600$   
$8x1 + 10x2 + 12x3 + s3 = 600$

In [37]:
import numpy as np
z = np.array([-4.5, -5,-4,0,0,0])
A = np.array([[10,5,6,1,0,0],
            [6,8,9,0,1,0],
            [8,10,12,0,0,1]])
b = np.array([600,600,600,0])


# Etiquetar las variables
variables = ['x1', 'x2', 'x3', 's1', 's2', 's3']  # Variables iniciales

# Inicializar variables básicas y no básicas
variables_básicas = ['s1', 's2', 's3']  # Variables de holgura inicialmente básicas
variables_no_básicas = ['x1', 'x2', 'x3']  # Variables originales no básicas


In [39]:
def tabla_simplex(z, A, b):
    return np.hstack((np.vstack((A,z)),np.vstack(b)))

tabla_simplex(z,A,b)

array([[ 10. ,   5. ,   6. ,   1. ,   0. ,   0. , 600. ],
       [  6. ,   8. ,   9. ,   0. ,   1. ,   0. , 600. ],
       [  8. ,  10. ,  12. ,   0. ,   0. ,   1. , 600. ],
       [ -4.5,  -5. ,  -4. ,   0. ,   0. ,   0. ,   0. ]])

In [30]:
def encontrar_columna_pivote(z):
    val = np.min(z)
    col = np.argmin(z)
    return val, col

#val, col = encontrar_columna_pivote(z)
#print(val, col)

In [31]:
def encontrar_fila_pivote(A, b, col):
    filas, columnas = np.shape(A)
    ratio = []

    for i in range(filas):
        ratio_val = b[i]/A[i,col]
        if ratio_val > 0:
            ratio.append(ratio_val)

    return ratio.index(min(ratio))

#fil = encontrar_fila_pivote(A, b, col)

In [32]:
def actualizar_tabla(z, A, b, fil, col):
    filas, columnas = np.shape(A)
    A = A.astype(float)

    # Obtener un 1 en la intersección de la fila y columna pivote
    pivote = A[fil,col]
    A[fil] = A[fil]/pivote
    b[fil] = b[fil]/pivote

    # Actualizar los valores de las ecuaciones
    for i in range(filas):
        if A[i,col] != 0 and i != fil:
            b[i] = b[i] - (b[fil]*A[i,col])
            A[i] = A[i] - (A[fil]*A[i,col])

    # Actualizar la función objetivo
    b[filas] = b[filas] - (b[fil]*z[col])
    z = z - (A[fil]*z[col])

    return z, A, b

#actualizar_tabla(z, A, b, fil, col)

In [33]:
# Mostrar la tabla inicial
print("Tabla inicial (Simplex):")
print(tabla_simplex(z, A, b))
print(f"Variables básicas: {variables_básicas}, Variables no básicas: {variables_no_básicas}")

val, col = encontrar_columna_pivote(z)
iteracion = 1

# Bucle Simplex
while val < 0:
    print(f"\nIteración {iteracion}:")

    # Encontrar la fila pivote
    fil = encontrar_fila_pivote(A, b, col)
    print(f"Fila pivote: {fil}, Columna pivote: {col} (Variable entrante: {variables_no_básicas[col]})")
    
    # Mostrar qué variable entra y qué variable sale
    var_saliente = variables_básicas[fil]
    var_entrante = variables_no_básicas[col]
    print(f"Variable {var_saliente} sale de la base y la variable {var_entrante} entra en la base.")
    
    # Actualizar las variables básicas y no básicas
    variables_básicas[fil] = var_entrante
    
    # Actualizar la tabla con el pivote
    z, A, b = actualizar_tabla(z, A, b, fil, col)
    
    # Mostrar la tabla actualizada
    print("\nTabla actualizada:")
    print(tabla_simplex(z, A, b))
    print(f"Variables básicas: {variables_básicas}, Variables no básicas: {variables_no_básicas}")
    
    # Encontrar la nueva columna pivote
    val, col = encontrar_columna_pivote(z)
    iteracion += 1

# Mostrar la tabla final y solución
print("\nTabla final (solución óptima):")
print(tabla_simplex(z, A, b))
# Calcular los valores finales de las variables
solucion = np.zeros(len(variables))
for i in range(len(variables_básicas)):
    indice = variables.index(variables_básicas[i])
    solucion[indice] = b[i]
    
# Mostrar la solución final
print("\nSolución final:")
for i, valor in enumerate(solucion):
    print(f"{variables[i]} = {valor}")
print(f"Valor óptimo de la función objetivo: {-z[-1]}")

Tabla inicial (Simplex):
[[ 10.    5.    6.    1.    0.    0.  600. ]
 [  6.    8.    9.    0.    1.    0.  600. ]
 [  8.   10.   12.    0.    0.    1.  600. ]
 [ -4.5  -5.   -4.    0.    0.    0.    0. ]]
Variables básicas: ['s1', 's2', 's3'], Variables no básicas: ['x1', 'x2', 'x3']

Iteración 1:
Fila pivote: 2, Columna pivote: 1 (Variable entrante: x2)
Variable s3 sale de la base y la variable x2 entra en la base.

Tabla actualizada:
[[ 6.0e+00  0.0e+00  0.0e+00  1.0e+00  0.0e+00 -5.0e-01  3.0e+02]
 [-4.0e-01  0.0e+00 -6.0e-01  0.0e+00  1.0e+00 -8.0e-01  1.2e+02]
 [ 8.0e-01  1.0e+00  1.2e+00  0.0e+00  0.0e+00  1.0e-01  6.0e+01]
 [-5.0e-01  0.0e+00  2.0e+00  0.0e+00  0.0e+00  5.0e-01  3.0e+02]]
Variables básicas: ['s1', 's2', 'x2'], Variables no básicas: ['x1', 'x2', 'x3']

Iteración 2:
Fila pivote: 0, Columna pivote: 0 (Variable entrante: x1)
Variable s1 sale de la base y la variable x1 entra en la base.

Tabla actualizada:
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00  1.666666

In [34]:
from scipy.optimize import linprog
z = np.array([-4.5, -5, -4])
A = np.array([[10, 5, 6], [6, 8, 9], [8, 10, 12]])
b = np.array([600, 600, 600])

In [35]:
resul = linprog(z,A,b,method="revised simplex")
print(resul)


 message: Optimization terminated successfully.
 success: True
  status: 0
     fun: -325.0
       x: [ 5.000e+01  2.000e+01  0.000e+00]
     nit: 2


C:\Users\Danie\AppData\Local\Temp\ipykernel_13236\1680923369.py:1: DeprecationWarning: `method='revised simplex'` is deprecated and will be removed in SciPy 1.11.0. Please use one of the HiGHS solvers (e.g. `method='highs'`) in new code.
  resul = linprog(z,A,b,method="revised simplex")


In [36]:
    print('z_max = {0:.1f}, x1 = {1:.1f}, x2 = {2:.1f}, x3 = {3:.1f}'.
      format(-resul.fun,resul.x[0],resul.x[1],resul.x[2]))

z_max = 325.0, x1 = 50.0, x2 = 20.0, x3 = 0.0
